In [1]:
import numpy as np
from skimage.transform import resize
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, ConvLSTM2D, TimeDistributed
from sklearn.externals import joblib
import pylidc as pl

/usr/share/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
image_size = (256, 256)
num_slices = 50

In [3]:
model = Sequential()
model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3), input_shape=(num_slices, *image_size, 1),
                     padding='same', return_sequences=True))
model.add(TimeDistributed(Conv2D(1, 1, activation='sigmoid')))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
unet = load_model('unet.hdf5')
model = load_model('unet_lstm.hdf5')

In [4]:
scans = pl.query(pl.Scan)
n = scans.count()

In [5]:
test = joblib.load('test_set.p')
train = list(set(range(n)) - set(test))

In [6]:
def get_annotated_slices(ann):
    zs = ann.bbox()[-1]
    return zs.start, zs.stop

In [14]:
def get_data(i, draws):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([resize(im.pixel_array, image_size, mode='constant') for im in images])
    X = np.expand_dims(X, axis=-1)
    y = np.zeros((512, 512, n))
    for ann in scan.annotations:
        b = ann.bbox()
        y[b] = np.logical_or(y[b], ann.boolean_mask())
    y = np.rollaxis(y, 2, 0)
    y = resize(y, (n, *image_size), mode='constant')
    y = np.expand_dims(y, axis=-1)
    nodule_zs = set.union(set(), *(set(range(*get_annotated_slices(ann))) for ann in scan.annotations))
    X_acc, y_acc = [], []
    for _ in range(draws):
        while True:
            start = np.random.randint(n - num_slices)
            if any(start <= z < start + num_slices for z in nodule_zs) or not nodule_zs:
                break
        X_acc.append(unet.predict(X[start:start + num_slices], batch_size=1))
        y_acc.append(y[start:start + num_slices])
    return np.array(X_acc), np.array(y_acc)

In [11]:
num_epochs = 2
num_draws = 4
num_samples = 200

In [ ]:
for _ in range(num_samples):
    while True:
        i = np.random.choice(train)
        print(i)
        try:
            X, y = get_data(i, num_draws)
            model.fit(X, y, epochs=num_epochs, batch_size=1)
            model.save('unet_lstm.hdf5')
            break
        except OSError:
            print('Problem getting data')

14
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0259 - acc: 1.0000
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0257 - acc: 1.0000
914
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0257 - acc: 0.9999
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0254 - acc: 0.9999
510
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0251 - acc: 1.0000
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0249 - acc: 1.0000
252
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0244 - acc: 1.0000
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0242 - acc: 1.0000
893
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0246 - acc: 0.9999
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0244 - acc: 0.9999
381
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0239 - acc: 0.999

4/4 [==============================] - 4s 1s/step - loss: 0.0141 - acc: 1.0000
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0140 - acc: 1.0000
668
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0143 - acc: 0.9999
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0142 - acc: 0.9999
394
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0137 - acc: 1.0000
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0136 - acc: 1.0000
784
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0145 - acc: 0.9998
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0144 - acc: 0.9998
624
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0139 - acc: 0.9999
Epoch 2/2
4/4 [==============================] - 4s 1s/step - loss: 0.0138 - acc: 0.9999
115
Epoch 1/2
4/4 [==============================] - 4s 1s/step - loss: 0.0141 - acc: 0.9999
Epoch 2/2
4